In [ ]:
import nltk
from konlpy.tag import *
from ckonlpy.tag import Twitter
import MeCab
import kss
from collections import Counter
import tensorflow as tf
import numpy as np
import re
from soynlp.normalizer import *
import hanspell
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.datasets import fetch_20newsgroups
import pandas as pd
from math import log
import urllib
import matplotlib as mpl
import matplotlib.pyplot as plt
import json
import os
from pprint import pprint
import platform
from tqdm.notebook import tqdm

plt.style.use("dark_background")

okt = Okt()
kkm = Kkma()
kmr = Komoran()
hnn = Hannanum()
twt = Twitter()

class Mecab:
    def pos(self, text):
        p = re.compile(".+\t[A-Z]+")
        return [tuple(p.match(line).group().split("\t")) for line in MeCab.Tagger().parse(text).splitlines()[:-1]]
    
    def morphs(self, text):
        p = re.compile(".+\t[A-Z]+")
        return [p.match(line).group().split("\t")[0] for line in MeCab.Tagger().parse(text).splitlines()[:-1]]
    
    def nouns(self, text):
        p = re.compile(".+\t[A-Z]+")
        temp = [tuple(p.match(line).group().split("\t")) for line in MeCab.Tagger().parse(text).splitlines()[:-1]]
        nouns=[]
        for word in temp:
            if word[1] in ["NNG", "NNP", "NNB", "NNBC", "NP", "NR"]:
                nouns.append(word[0])
        return nouns
    
mc = Mecab()

nltk.download("punkt")
nltk.download("stopwords")
nltk.download("wordnet")

path = "C:/Windows/Fonts/malgun.ttf"
if platform.system() == "Darwin":
    mpl.rc("font", family="AppleGothic")
elif platform.system() == "Windows":
    font_name = mpl.font_manager.FontProperties(fname=path).get_name()
    mpl.rc('font', family=font_name)
    
mpl.rc("axes", unicode_minus=False)

C:\Users\5CG7092POZ\Anaconda3\envs\tf2.0\lib\site-packages\konlpy\tag\_okt.py:16: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\5CG7092POZ\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\5CG7092POZ\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\5CG7092POZ\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

# 1. Bag of Words
- Bag of Words란 단어들의 순서는 전혀 고려하지 않고, 단어들의 출현 빈도(frequency)에만 집중하는 텍스트 데이터의 수치화 표현 방법입니다. Bag of Words를 직역하면 단어들의 가방이라는 의미입니다. 단어들이 들어있는 가방을 상상해봅시다. 갖고있는 어떤 텍스트 문서에 있는 단어들을 가방에다가 전부 넣습니다. 그러고나서 이 가방을 흔들어 단어들을 섞습니다. 만약, 해당 문서 내에서 특정 단어가 N번 등장했다면, 이 가방에는 그 특정 단어가 N개 있게됩니다. 또한 가방을 흔들어서 단어를 섞었기 때문에 더 이상 단어의 순서는 중요하지 않습니다.

## 1-1. 직접 구현하기

In [ ]:
#정규 표현식을 통해 온점을 제거하는 정제 작업입니다.
token = re.sub("(\.)","","정부가 발표하는 물가상승률과 소비자가 느끼는 물가상승률은 다르다.")  
token = okt.morphs(token)  

word2idx = {}
bow = []
for voca in token:
    if voca not in word2idx.keys():
        # token을 읽으면서, word2idx에 없는 (not in) 단어는 새로 추가하고, 이미 있는 단어는 넘깁니다.
        word2idx[voca] = len(word2idx)
# BoW 전체에 전부 기본값 1을 넣어줍니다. 단어의 개수는 최소 1개 이상이기 때문입니다.
        bow.insert(-1, 1)
    else:
        #재등장하는 단어의 인덱스를 받아옵니다.
        idx = word2idx.get(voca)
        #재등장한 단어는 해당하는 인덱스의 위치에 1을 더해줍니다.(단어의 개수를 세는 것입니다.)
        bow[idx] = bow[idx] + 1

print(word2idx)
print(bow)

{'정부': 0, '가': 1, '발표': 2, '하는': 3, '물가상승률': 4, '과': 5, '소비자': 6, '느끼는': 7, '은': 8, '다르다': 9}
[1, 2, 1, 1, 2, 1, 1, 1, 1, 1]


## 1-2. CountVectorizer 사용하기

In [ ]:
corpus = ["you know I want your love. because I love you."]
vect = CountVectorizer()

#코퍼스로부터 각 단어의 빈도 수를 기록한다.
print(vect.fit_transform(corpus).toarray())
#각 단어의 인덱스가 어떻게 부여되었는지를 보여준다.
print(vect.vocabulary_)

[[1 1 2 1 2 1]]
{'you': 4, 'know': 1, 'want': 3, 'your': 5, 'love': 2, 'because': 0}


# 2. TF-IDF

## 2-1. 직접 구현하기

In [ ]:
docs = ["먹고 싶은 사과", "먹고 싶은 바나나", "길고 노란 바나나 바나나", "저는 과일이 좋아요"] 
vocab = list(set(word for doc in docs for word in doc.split(" ")))
vocab.sort()

In [ ]:
vocab

['과일이', '길고', '노란', '먹고', '바나나', '사과', '싶은', '저는', '좋아요']

### (1) TF

In [ ]:
#총 문서의 수
N = len(docs)

def calc_tf(term, doc):
    return doc.count(term)

def calc_idf(term):
    df = 0
    for doc in docs:
        df += term in doc
    return log(N/(df + 1))

def calc_tfidf(term, doc):
    return calc_tf(term, doc)*calc_idf(term)

tf = []
for i in range(N):
    tf.append([])
    doc = docs[i]
    for j in range(len(vocab)):
        term = vocab[j]
        tf[-1].append(calc_tf(term, doc))

tf_df = pd.DataFrame(tf, columns=vocab)

In [ ]:
tf_df

,과일이,길고,노란,먹고,바나나,사과,싶은,저는,좋아요
0,0,0,0,1,0,1,1,0,0
1,0,0,0,1,1,0,1,0,0
2,0,1,1,0,2,0,0,0,0
3,1,0,0,0,0,0,0,1,1


### (2) IDF

In [ ]:
idf = []
for i in range(len(vocab)):
    term = vocab[i]
    idf.append(calc_idf(term))
idf_ser = pd.Series(idf, index=vocab)

In [ ]:
idf_ser

과일이    0.693147
길고     0.693147
노란     0.693147
먹고     0.287682
바나나    0.287682
사과     0.693147
싶은     0.287682
저는     0.693147
좋아요    0.693147
dtype: float64

### (3) TF-IDF

In [ ]:
tfidf = []
for i in range(N):
    doc = docs[i]
    tfidf.append([])
    for j in range(len(vocab)):
        term = vocab[j]
        tfidf[-1].append(calc_tfidf(term, doc))
tfidf_df = pd.DataFrame(tfidf, columns=vocab)
tfidf_df

,과일이,길고,노란,먹고,바나나,사과,싶은,저는,좋아요
0,0.000000,0.000000,0.000000,0.287682,0.000000,0.693147,0.287682,0.000000,0.000000
1,0.000000,0.000000,0.000000,0.287682,0.287682,0.000000,0.287682,0.000000,0.000000
2,0.000000,0.693147,0.693147,0.000000,0.575364,0.000000,0.000000,0.000000,0.000000
3,0.693147,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.693147,0.693147


- 지금까지 TF-IDF의 가장 기본적인 식에 대해서 학습하고, 이를 실제로 구현하는 실습을 진행해보았습니다. 그런데 사실 실제 TF-IDF 구현을 제공하고 있는 많은 패키지들은 패키지마다 식이 조금씩 다르긴 하지만, 위에서 배운 기본 식에서 조정된 식을 사용합니다. 그 이유는 위의 기본적인 식을 바탕으로 한 구현에도 여전히 문제점이 존재하기 때문입니다. 만약 전체 문서의 수 `N=4`인데, `DF=3`인 경우에는 어떤 일이 벌어질까요? `IDF=0`이 됨을 의미합니다. 식으로 표현하면 `IDF = log(N/(DF + 1)) = 0`입니다. IDF의 값이 0이라면 더 이상 가중치의 역할을 수행하지 못합니다. 그래서 실제 구현체는 IDF = log(N/(DF + 1)) + 1과 같이 log항에 1을 더해줘서 log항의 값이 0이 되더라도 IDF가 최소 1이상의 값을 가지도록 합니다. 사이킷런도 이 방식을 사용합니다.

## 1-2. Tensorflow로 구현하기

In [ ]:
texts = ["먹고 싶은 사과", "먹고 싶은 바나나", "길고 노란 바나나 바나나", "저는 과일이 좋아요"]
tkn = tf.keras.preprocessing.text.Tokenizer()
tkn.fit_on_texts(texts)

In [ ]:
print(tkn.word_index)

{'바나나': 1, '먹고': 2, '싶은': 3, '사과': 4, '길고': 5, '노란': 6, '저는': 7, '과일이': 8, '좋아요': 9}


- 각 단어에 숫자 1부터 시작하는 정수 인덱스가 부여되었습니다. 이제 텍스트 데이터에 `texts_to_matrix()`를 사용해보겠습니다. `texts_to_matrix()`란 이름에서 알 수 있지만, 이 도구는 입력된 텍스트 데이터로부터 행렬(matrix)를 만드는 도구입니다. 총 4개의 모드를 지원하는데 각 모드는 `"count"`, `"binary"`, `"tfidf"`, `"freq"`입니다. 우선 `"count"` 모드를 사용해봅시다.

### (1) `mode="binary"`

In [ ]:
print(tkn.texts_to_matrix(texts, mode="binary"))

[[0. 0. 1. 1. 1. 0. 0. 0. 0. 0.]
 [0. 1. 1. 1. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 1. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 1. 1.]]


- DTM과 결과가 매우 유사해보입니다. 다만 세번째 행, 두번째 열의 값이 DTM에서는 2였는데 여기서는 1로 바뀌었습니다. 그 이유는 "binary" 모드는 해당 단어가 존재하는지만 관심을 가지고 해당 단어가 몇 개였는지는 무시하기 때문입니다. 해당 단어가 존재하면 1, 단어가 존재하지 않으면 0의 값을 가집니다. 즉, 단어의 존재 유무로만 행렬을 표현합니다.

### (2) `mode="count"`

In [ ]:
print(tkn.texts_to_matrix(texts, mode="count"))

[[0. 0. 1. 1. 1. 0. 0. 0. 0. 0.]
 [0. 1. 1. 1. 0. 0. 0. 0. 0. 0.]
 [0. 2. 0. 0. 0. 1. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 1. 1.]]


- "count"를 사용하면 우리가 앞서 배운 문서 단어 행렬(Document-Term Matrix, DTM)을 생성합니다. DTM에서의 인덱스는 앞서 확인한 word_index의 결과입니다.
- 다만 주의할 점은 각 단어에 부여되는 인덱스는 1부터 시작하는 반면에 완성되는 행렬의 인덱스는 0부터 시작합니다. 실제로 단어의 개수는 9개였지만 완성된 행렬의 열의 개수는 10개인 것과 첫번째 열은 모든 행에서 값이 0인 것을 볼 수 있습니다. 인덱스 0에는 그 어떤 단어도 할당되지 않았기 때문입니다.
- 네번째 행을 보겠습니다. 네번째 행은 테스트 데이터에서 네번째 문장을 의미합니다. 네번째 행은 8번째 열, 9번째 열, 10번째 열에서 1의 값을 가집니다. 이는 7번 단어, 8번 단어, 9번 단어가 네번째 문장에서 1개씩 존재함을 의미합니다. 위에서 정수 인코딩 된 결과를 보면 7번 단어는 "저는", 8번 단어는 "과일이", 9번 단어는 "좋아요"입니다. 세번째 행의 첫번째 열의 값은 2인데, 이는 세번째 문장에서 1번 인덱스를 가진 바나나가 두 번 등장했기 때문입니다.
- 앞서 배웠듯이 DTM은 bag of words를 기반으로 하므로 단어 순서 정보는 보존되지 않습니다. 사실 더 구체적으로는 4개의 모든 모드에서 단어 순서 정보는 보존되지 않습니다.

### (3) `mode="tfidf"`

In [ ]:
#둘째 자리까지 반올림하여 출력합니다.
print(tkn.texts_to_matrix(texts, mode="tfidf").round(3))

[[0.    0.    0.847 0.847 1.099 0.    0.    0.    0.    0.   ]
 [0.    0.847 0.847 0.847 0.    0.    0.    0.    0.    0.   ]
 [0.    1.435 0.    0.    0.    1.099 1.099 0.    0.    0.   ]
 [0.    0.    0.    0.    0.    0.    0.    1.099 1.099 1.099]]


- 말 그대로 TF-IDF 행렬을 만듭니다. 다만, TF-IDF 챕터에서 배운 기본식이나 사이킷런의 TfidfVectorizer에서 사용하는 식이랑 또 조금 다릅니다. TF를 각 문서에서의 각 단어의 빈도에 자연 로그를 씌우고 1을 더한 값으로 정의했습니다. IDF에서는 앞서 배운 기본식에서 로그는 자연 로그를 사용하고, 로그 안의 분수에 1을 추가로 더했습니다. 물론, 이러한 식을 굳이 기억할 필요는 없고 여전히 TF-IDF의 기존 의도를 갖고 있다고 이해하면 됩니다.

### (4) `mode="freq"`

In [ ]:
#둘째 자리까지 반올림하여 출력합니다.
print(tkn.texts_to_matrix(texts, mode="freq").round(3))

[[0.    0.    0.333 0.333 0.333 0.    0.    0.    0.    0.   ]
 [0.    0.333 0.333 0.333 0.    0.    0.    0.    0.    0.   ]
 [0.    0.5   0.    0.    0.    0.25  0.25  0.    0.    0.   ]
 [0.    0.    0.    0.    0.    0.    0.    0.333 0.333 0.333]]


- 각 문서에서의 각 단어의 등장 횟수를 분자로, 각 문서의 크기(각 문서에서 등장한 모든 단어의 개수의 총 합)를 분모로 하는 표현 방법입니다. 예를 들어 세번째 행을 보겠습니다. 세번째 문장은 "길고 노란 바나나 바나나" 였습니다. 문서의 크기는 4인데, 바나나는 총 2회 등장했습니다. 이에 따라서 세번째 문장에서의 단어 "바나나"의 값은 위의 행렬에서 0.5가 됩니다. 반면에 "길고", "노란"이라는 두 단어는 각 1회 등장했으므로 각자 1/4의 값인 0.25의 값을 가집니다.